In this homework, we'll deploy the Bank Marketing model from the homework 5. We already have a docker image for this model - we'll use it for deploying the model to Kubernetes. 

### Question 1: Deploy the model to Kubernetes

The command to run the docker container is:
``` bash
docker pull svizor/zoomcamp-model:3.11.5-hw10
docker run -it -p 9696:9696 svizor/zoomcamp-model:3.11.5-hw10
```


To test the model we can use the following code:
``` python 
python q6_test.py


This is the output of the test:
```
python q6_test.py
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
```

### Question 2

In [1]:
!powershell kind version

kind v0.25.0 go1.22.9 windows/amd64


In [ ]:
!powershell kind create cluster  # create a cluster with the default name

ERROR: failed to create cluster: node(s) already exist for a cluster with the name "kind"


In [3]:
!powershell kubectl cluster-info # check if the cluster is running 

Kubernetes control plane is running at https://127.0.0.1:52651
CoreDNS is running at https://127.0.0.1:52651/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


### Question 3

The smallest deployment computing unit in kubernetes is a pod. A pod is a group of one or more containers, with shared storage/network resources, and a specification for how to run the containers.

### Question 4

In [7]:
!powershell kubectl get service # check the services running in the cluster

NAME           TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1      <none>        443/TCP        4h46m
subscription   LoadBalancer   10.96.30.244   <pending>     80:30091/TCP   4h19m


### Question 5

In [8]:
!powershell kind load docker-image zoomcamp-model:3.11.5-hw10 # load the docker image into the cluster

Image: "zoomcamp-model:3.11.5-hw10" with ID "sha256:020904a2523cec81c854a9b4679ae26d23f814e42350721136d1447f910a6b53" found to be already present on all nodes.


### Question 6

image: zoomcamp-model:3.11.5-hw10 # the image name \
memory: "128Mi" # 128 mebibytes of memory\
cpu: "200m" # 0.2 of a core\
port : 9696 # the port the model is running on

In [9]:
!powershell kubectl apply -f deployment.yaml # apply the deployment.yaml file to the cluster

deployment.apps/subscription unchanged


In [10]:
!powershell kubectl get pods # check the pods running in the cluster

NAME                           READY   STATUS    RESTARTS      AGE
subscription-7b74b5846-p47hz   1/1     Running   1 (73s ago)   4h


### Question 7

In [11]:
!powershell kubectl apply -f service.yaml # apply the service.yaml file to the cluster

service/subscription unchanged


In [12]:
!powershell kubectl get pods # check the pods running in the cluster

NAME                           READY   STATUS    RESTARTS      AGE
subscription-7b74b5846-p47hz   1/1     Running   1 (80s ago)   4h


In [13]:
!powershell kubectl port-forward pod/subscription-7b74b5846-p47hz 9696:9696 # port forward the pod to the localhost

^C


Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696


In [14]:
!python q6_test.py # run the test script to check if the service is working correctly

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


This is the same result as the one we got above when we ran just the docker container. 

### Followup HPA

In [15]:
!powershell kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3 # autoscale the deployment

Error from server (AlreadyExists): horizontalpodautoscalers.autoscaling "subscription-hpa" already exists


In [16]:
!powershell kubectl get hpa # check the horizontal pod autoscaler

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         1          179m


In [21]:
!powershell kubectl get pods -n kube-system # check the pods running in the kube-system namespace


NAME                                         READY   STATUS    RESTARTS        AGE
coredns-7c65d6cfc9-6zrx8                     1/1     Running   1 (4m29s ago)   4h49m
coredns-7c65d6cfc9-s94tr                     1/1     Running   1 (4m29s ago)   4h49m
etcd-kind-control-plane                      1/1     Running   1 (4m29s ago)   4h49m
kindnet-pjxks                                1/1     Running   1 (4m29s ago)   4h49m
kube-apiserver-kind-control-plane            1/1     Running   1 (4m29s ago)   4h49m
kube-controller-manager-kind-control-plane   1/1     Running   1 (4m29s ago)   4h49m
kube-proxy-xffh8                             1/1     Running   1 (4m29s ago)   4h49m
kube-scheduler-kind-control-plane            1/1     Running   1 (4m29s ago)   4h49m
metrics-server-587b667b55-jk6k9              1/1     Running   0               81s


In [18]:
!powershell kubectl patch deployment metrics-server -n kube-system --type='json' -p='[{"op": "add", "path": "/spec/template/spec/containers/0/args/-", "value": "--kubelet-insecure-tls"}]'

deployment.apps/metrics-server patched


In [22]:
!powershell kubectl get hpa # check the horizontal pod autoscaler

NAME               REFERENCE                 TARGETS       MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: 1%/20%   1         3         1          3h1m


This shows that the horizontal pod autoscaler and the metrics server are working correctly.  We will now try to test the autoscaler by sending a lot of requests to the model.

### Question 8

In [24]:
!python q6_test_hpa.py # run the test script to check if the service is working correctly with the horizontal pod autoscaler

^C


In [25]:
!powershell kubectl get hpa subscription-hpa --watch # check the horizontal pod autoscaler

^C


I could get the hpa to work by sending a lot of requests to the model using the hpa test script python file . Unfortunately it never needed to scale up because the model was able to handle the requests without any issues. 